In [ ]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import colorcet as cc
import cartopy.feature as cf 
import cartopy.crs as ccrs
from vis_tools import read_hysplit_netcdf

# ignore warning that arises from my use of NaN's instead of 0's
import warnings
warnings.filterwarnings('ignore', message='invalid value encountered in less_equal')

##############################################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '/opt/hysplit/18040212_ruapehu_12.5_0.01.nc'

# SPECIFY: pick one time step to plot
TIME_STEP = 1

# SPECIFY: plotting params
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
CONTOURS = None  # number of contours (or specify None)
##############################################################

model = read_hysplit_netcdf(FILENAME)

NZTM_PROJ = ccrs.epsg(2193)  # define projection using NZTM EPSG code (need internet access)

# create figure and set font size
plt.figure(figsize=(10, 8))
mpl.rcParams.update({'font.size': 14})
ax = plt.axes(projection=NZTM_PROJ)

# PLOT background geography
ax.coastlines(resolution='10m', zorder=3)
ax.add_feature(cf.NaturalEarthFeature(category='physical', scale='10m', name='lakes', edgecolor='k', facecolor='none', zorder=4))

# PLOT ashfall (using log scale)
ash = model.isel(time=TIME_STEP).total_deposition
cbar_kwargs = dict(label='ash thickness (mm)', extend='neither')
ash.plot.pcolormesh(transform=ccrs.PlateCarree(), cmap=cc.m_fire_r, cbar_kwargs=cbar_kwargs, norm=colors.LogNorm(),\
                    vmin=ASH_MIN, vmax=ASH_MAX, zorder=1)
if CONTOURS != None:
    ash.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=np.logspace(np.log10(ASH_MIN), np.log10(ASH_MAX), CONTOURS),\
                     colors='k', alpha=0.5, linewidths=0.75, zorder=2)

# PLOT source location
src_lat = model.attrs['volcano_location'][0]
src_lon = model.attrs['volcano_location'][1]
ax.plot(src_lon, src_lat, transform=ccrs.Geodetic(), marker='^', ms=10, mew=0.75, c='c', mec='k', zorder=6)

ts = int(np.unique(np.ediff1d(model['time'])).astype('timedelta64[h]').astype(int))  # hours

plt.title(FILENAME.split('/')[-1] + '\n' + str(TIME_STEP*ts) + ' hours after eruption onset');